In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
from sys import exit
from matplotlib.backends.backend_pdf import PdfPages 
import matplotlib.dates as mdates
import seaborn as sns
from calendar import monthrange
import datetime
from tqdm import tqdm
import ast
import os
from scipy import stats

In [ ]:
# This is a script copy of "Script10.py", ported to JupyterNotebook for ease.

In [ ]:
# RESAMPLE datasets at varying frequency and plot various data.
def Create_PDFs(Filename,Identifier): 
    p = PdfPages(Filename) 
    fig_nums = plt.get_fignums()   
    figs = [plt.figure(n) for n in fig_nums] 
    for fig in tqdm(figs, desc=f'  Plotting {Identifier} ...'):  
        fig.savefig(p, format='pdf')  
    p.close()
#
def GetPropertyDetails(DataRow):
    EPCRating = DataRow['House_SAP'].values[0]
    FloorArea = DataRow['Total_Floor_Area'].values[0]
    Employment = DataRow['EmployType'].values[0]
    HouseIncome = DataRow['HouseIncome'].values[0]
    Tenure = DataRow['Tenure'].values[0]
    HPType = DataRow['HP_Installed'].values[0]
    HP_Size_kW = DataRow['HP_Size_kW'].values[0]
    Age = DataRow['Participant_Age'].values[0]
    JobStatus = DataRow['EmployStatus'].values[0]
    Fluid = DataRow['HP_Refrigerant'].values[0]
    MCS_SHLoad = DataRow['MCS_SHLoad'].values[0]
    MCS_Hloss = DataRow['MCS_Hloss'].values[0]
    return(EPCRating,FloorArea,Employment,HouseIncome,Tenure,HPType,HP_Size_kW,Age,JobStatus,Fluid,MCS_SHLoad,MCS_Hloss)
#
def GetPolyCoefficients(XValues,YValues):
    coeff_A1,YIntercpt_A1 = np.polyfit(XValues,YValues,deg=1) # linear plot
    LinearFit = coeff_A1*X + YIntercpt_A1
    coeff_B1,coeff_B2,YIntercptB = np.polyfit(XValues,YValues,deg=2) # polynominal plot.
    PolyFit = coeff_B1*X*X + coeff_B2*X + YIntercptB
    return(LinearFit,coeff_A1,YIntercpt_A1,PolyFit,coeff_B1,coeff_B2,YIntercptB)

In [ ]:
Month = {1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',
         7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'}
Seasons = {1:'Winter',2:'Winter',3:'Spring',4:'Spring',5:'Spring',6:'Summer',
         7:'Summer',8:'Summer',9:'Autumn',10:'Autumn',11:'Autumn',12:'Winter'}
DayName = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
WeekDayWeekend = {0:'Weekday',1:'Weekday',2:'Weekday',3:'Weekday',4:'Weekday',5:'WeekEnd',6:'WeekEnd'}

In [ ]:
SampleRate = '60min'

In [ ]:
SummaryTable = pd.read_csv("../Data/UK Data Service/Electrification of Heat Demonstration Project 2020-2022/BEIS Electrification of Heat Project - Interim Heat Pump Performance Data Summary.csv")
PropDesignTbl = pd.read_csv("../Data/UK Data Service/Electrification of Heat Demonstration Project 2020-2022/BEIS Electrification of Heat Project - Property, Design and Installation Information.csv")
SummaryTable = SummaryTable.loc[(SummaryTable['Included_SPF_analysis'] == True)]
ValidPropertyIDs = SummaryTable['Property_ID'].tolist()
PropertyList = tqdm(ValidPropertyIDs)
ValidList1 = []
MasterDF = pd.DataFrame()
LogFile = open(f"LogFile_{SampleRate}.Log",mode="w")
#
for Propertyname in PropertyList:
    PropertyList.set_description(f'Calculating ({Propertyname}) ...')
    Filename = f"Property_ID={Propertyname}.csv"
    FilePath1 = f"../Data/UK Data Service/Electrification of Heat Demonstration Project 2020-2022/COMBINED/{Filename}"
    if os.path.exists(FilePath1) is False:
        exit(f"{FilePath1} didn't exist.\nMoving to the next ...")
    Table = pd.read_csv(FilePath1)
    Columns = Table.columns.tolist()
    ColumCalcList = ["Back-up_Heater_Energy_Consumed","Immersion_Heater_Energy_Consumed","Circulation_Pump_Energy_Consumed"]
    if "Boiler_Energy_Output" in Columns:
        print(f"EXCLUDED - {Propertyname} Hybrid system.",file=LogFile)
        pass
    elif "Brine_Flow_Temperature" in Columns:
        print(f"EXCLUDED - {Propertyname} GSHP system.",file=LogFile)
        pass
    else:
        ValidList1.append(Propertyname)
        #Table.drop(columns=['Heat_Pump_Heating_Flow_Temperature', 'Heat_Pump_Return_Temperature','Hot_Water_Flow_Temperature'],inplace=True)
        Table['Timestamp']= pd.to_datetime(Table['Timestamp'])
        Table = Table.resample(SampleRate, on='Timestamp').mean()
        Table.reset_index(inplace=True)
        Table = Table.loc[(Table['Whole_System_Energy_Consumed'] > 0.1) & (Table['Heat_Pump_Energy_Output'] > 0.1)]
        Table['Temperature_Makeup'] = abs(Table['Internal_Air_Temperature'] - Table['External_Air_Temperature']) # dT across the building.
        Overlap = [i for i in ColumCalcList if i in Columns]
        print(f"{Propertyname} - {len(Overlap)} == {Columns}",file=LogFile)
        if len(Overlap) == 3:
            Table['Back_up_Heater_Energy_Consumed'] = Table['Back-up_Heater_Energy_Consumed'] # Change '-' to '_', for later.
            Table.drop(columns=['Back-up_Heater_Energy_Consumed'],inplace=True) # Not this column 'Back-up_Heater_Energy_Consumed' is not needed.
            Table['Hourly_Heat_Pump_Energy_Output'] = Table['Heat_Pump_Energy_Output'].diff().fillna(Table['Heat_Pump_Energy_Output'])
            Table['Hourly_Whole_System_Energy_Consumed'] = Table['Whole_System_Energy_Consumed'].diff().fillna(Table['Whole_System_Energy_Consumed'])
            Table['Hourly_Back_up_Heater_Energy_Consumed'] = Table['Back_up_Heater_Energy_Consumed'].diff().fillna(Table['Back_up_Heater_Energy_Consumed'])
            Table['Hourly_Immersion_Heater_Energy_Consumed'] = Table['Immersion_Heater_Energy_Consumed'].diff().fillna(Table['Immersion_Heater_Energy_Consumed'])
            Table['Hourly_Circulation_Pump_Energy_Consumed'] = Table['Circulation_Pump_Energy_Consumed'].diff().fillna(Table['Circulation_Pump_Energy_Consumed'])            
            Table = Table.eval('Hourly_Heat_Pump_Power_Consumed = Hourly_Whole_System_Energy_Consumed - Hourly_Back_up_Heater_Energy_Consumed - Hourly_Immersion_Heater_Energy_Consumed - Hourly_Circulation_Pump_Energy_Consumed')
            Table = Table.eval('COP_H2 = Hourly_Heat_Pump_Energy_Output / Hourly_Heat_Pump_Power_Consumed')
            Table = Table.eval('COP_H3 = (Hourly_Heat_Pump_Energy_Output) / (Hourly_Heat_Pump_Power_Consumed+Hourly_Back_up_Heater_Energy_Consumed+Hourly_Immersion_Heater_Energy_Consumed)')
            Table = Table.eval('COP_H4 = (Hourly_Heat_Pump_Energy_Output) / (Hourly_Heat_Pump_Power_Consumed+Hourly_Back_up_Heater_Energy_Consumed+Hourly_Immersion_Heater_Energy_Consumed+Hourly_Circulation_Pump_Energy_Consumed)')
        elif len(Overlap) == 2:
            # Doesn't have "Back_up_Heater_Energy_Consumed"
            Table['Hourly_Heat_Pump_Energy_Output'] = Table['Heat_Pump_Energy_Output'].diff().fillna(Table['Heat_Pump_Energy_Output'])
            Table['Hourly_Whole_System_Energy_Consumed'] = Table['Whole_System_Energy_Consumed'].diff().fillna(Table['Whole_System_Energy_Consumed'])
            Table['Hourly_Immersion_Heater_Energy_Consumed'] = Table['Immersion_Heater_Energy_Consumed'].diff().fillna(Table['Immersion_Heater_Energy_Consumed'])
            Table['Hourly_Circulation_Pump_Energy_Consumed'] = Table['Circulation_Pump_Energy_Consumed'].diff().fillna(Table['Circulation_Pump_Energy_Consumed']) 
            Table = Table.eval('Hourly_Heat_Pump_Power_Consumed = Hourly_Whole_System_Energy_Consumed - Hourly_Immersion_Heater_Energy_Consumed - Hourly_Circulation_Pump_Energy_Consumed')
            Table = Table.eval('COP_H2 = Hourly_Heat_Pump_Energy_Output / Hourly_Heat_Pump_Power_Consumed')
            Table = Table.eval('COP_H3 = (Hourly_Heat_Pump_Energy_Output) / (Hourly_Heat_Pump_Power_Consumed+Hourly_Immersion_Heater_Energy_Consumed)')
            Table = Table.eval('COP_H4 = (Hourly_Heat_Pump_Energy_Output) / (Hourly_Heat_Pump_Power_Consumed+Hourly_Immersion_Heater_Energy_Consumed+Hourly_Circulation_Pump_Energy_Consumed)')
        elif len(Overlap) == 1:
            # Doesn't have "Back_up_Heater_Energy_Consumed" and "Immersion_Heater_Energy_Consumed",
            Table['Hourly_Heat_Pump_Energy_Output'] = Table['Heat_Pump_Energy_Output'].diff().fillna(Table['Heat_Pump_Energy_Output'])
            Table['Hourly_Whole_System_Energy_Consumed'] = Table['Whole_System_Energy_Consumed'].diff().fillna(Table['Whole_System_Energy_Consumed'])
            Table['Hourly_Circulation_Pump_Energy_Consumed'] = Table['Circulation_Pump_Energy_Consumed'].diff().fillna(Table['Circulation_Pump_Energy_Consumed']) 
            Table = Table.eval('Hourly_Heat_Pump_Power_Consumed = Hourly_Whole_System_Energy_Consumed - Hourly_Circulation_Pump_Energy_Consumed')
            Table = Table.eval('COP_H2 = Hourly_Heat_Pump_Energy_Output / Hourly_Heat_Pump_Power_Consumed')
            Table = Table.eval('COP_H3 = (Hourly_Heat_Pump_Energy_Output) / (Hourly_Heat_Pump_Power_Consumed)')
            Table = Table.eval('COP_H4 = (Hourly_Heat_Pump_Energy_Output) / (Hourly_Heat_Pump_Power_Consumed+Hourly_Circulation_Pump_Energy_Consumed)')
        else:
            print(f"{Propertyname} ZERO Overlaps.")
            break
        Table = Table.loc[(Table['Hourly_Heat_Pump_Energy_Output'] < 18) # Remove any system power greater tham 18kWth
                          & (Table['Hourly_Heat_Pump_Power_Consumed'] > 0.1) # Remove any noisy data
                          & (Table['COP_H2'] > 1) & (Table['COP_H2'] < 8) # Remove unrealistic values.
                          & (Table['Temperature_Makeup'] > 0.5)] # Remove any noisy data
        #
        Table.drop(Table.tail(1).index,inplace = True) # Drop last row - Removes and big jumps in data.
        Table.drop(Table.head(1).index,inplace = True) # Drop first row - Removes and big jumps in data.
        #
        PropertyLine = PropDesignTbl.loc[(PropDesignTbl['Property_ID'] == Propertyname)]
        EPCRating,FloorArea,Employment,HouseIncome,Tenure,HPType,HP_Size_kW,Age,JobStatus,Fluid,MCS_SHLoad,MCS_Hloss = GetPropertyDetails(PropertyLine)
        Table['Property_ID'] = Propertyname
        Table['EPCRating'] = EPCRating
        Table['FloorArea'] = FloorArea
        Table['Employment'] = Employment
        Table['HouseIncome'] = HouseIncome
        Table['Tenure'] = Tenure
        Table['HPType'] = HPType
        Table['HP_Size_kW'] = HP_Size_kW
        Table['Age'] = Age
        Table['JobStatus'] = JobStatus
        Table['HouseIncome'] = HouseIncome
        Table['HP_Refrigerant'] = Fluid
        Table['MCS_SHLoad'] = MCS_SHLoad
        Table['MCS_Hloss'] = MCS_Hloss
        if EPCRating in ['A','B','C']:
            Table['MCS_Characteristics'] = 'Low MCS SH Load'
            Table['EPC_Category'] = 'ABC'
        elif EPCRating in ['D','E','F','G']:
            Table['MCS_Characteristics'] = 'High MCS SH Load'
            Table['EPC_Category'] = 'DEFG'
        else:
            Table['MCS_Characteristics'] = 'UnKnown MCS SH Load'
            Table['EPC_Category'] = 'Unknown designation'
        #
        Table['Year'],Table['Month'],Table['Day'],Table['WeekNumber'],Table['DaysOfTheYear'],Table['DayOfTheWeek'],Table['Hour'],Table['Minute'] = Table['Timestamp'].dt.year,Table['Timestamp'].dt.month,Table['Timestamp'].dt.day,Table['Timestamp'].dt.isocalendar().week,Table['Timestamp'].dt.dayofyear,Table['Timestamp'].dt.dayofweek,Table['Timestamp'].dt.hour,Table['Timestamp'].dt.minute
        Table['MonthName'] = Table['Month'].map(Month)
        Table['Seasons'] = Table['Month'].map(Seasons)
        Table['WeekDayName'] = Table['DayOfTheWeek'].map(DayName)
        Table['WeekDay_OR_Weekend'] = Table['DayOfTheWeek'].map(WeekDayWeekend)
        Table['HP_Hourly_Power_Thresh_0.1kWe'] = np.where(Table['Hourly_Heat_Pump_Power_Consumed'] > 0.1, '1_HP_HighPower', '2_HP_LowPower')
        Table['HP_CircPumpInService_Thresh_1Wth'] = np.where(Table['Hourly_Circulation_Pump_Energy_Consumed'] > 0.001, '1_CP_InService', '2_CP_OutOfService')
        Table['Heat_Pump_Return_Temperature_Switch'] = np.where(Table['Heat_Pump_Return_Temperature'] < 30, '1_HP_ReturnTemp<30', '2_HP_ReturnTemp>30')
        Table['Hot_Water_Flow_Temperature_Switch'] = np.where(Table['Hot_Water_Flow_Temperature'] < 60, '1_DHW_FlowTemp<60', '2_DHW_FlowTemp>60')
        MasterDF = pd.concat([MasterDF,Table],ignore_index=True,sort=False)
        #
        #FilePath2 = f"../Data/UK Data Service/Electrification of Heat Demonstration Project 2020-2022/Resampled_{SampleRate}/{Filename}"
        #Table.to_csv(FilePath2)
#
# Reorgnising Year/Month/Day/Hour/Minute in a nice little order, just after the 'Timestamp'.
MasterDF.insert(1,'Minute',MasterDF.pop('Minute'))
MasterDF.insert(1,'Hour',MasterDF.pop('Hour'))
MasterDF.insert(1,'WeekDayName',MasterDF.pop('WeekDayName'))
MasterDF.insert(1,'WeekDay_OR_Weekend',MasterDF.pop('WeekDay_OR_Weekend'))
MasterDF.insert(1,'DayOfTheWeek',MasterDF.pop('DayOfTheWeek'))
MasterDF.insert(1,'DaysOfTheYear',MasterDF.pop('DaysOfTheYear'))
MasterDF.insert(1,'WeekNumber',MasterDF.pop('WeekNumber'))
MasterDF.insert(1,'Day',MasterDF.pop('Day'))
MasterDF.insert(1,'Month',MasterDF.pop('Month'))
MasterDF.insert(1,'Year',MasterDF.pop('Year'))
MasterDF.insert(1,'Seasons',MasterDF.pop('Seasons'))
MasterDF['ImmersionHeater_InService_Thresh_1Wth'] = np.where(MasterDF['Hourly_Immersion_Heater_Energy_Consumed'] > 0.001, '1_ImmerHeater_InService', '2_ImmerHeater_OutOfService')
MasterDF['BackUp_Heater_InService_Thresh_1Wth'] = np.where(MasterDF['Hourly_Back_up_Heater_Energy_Consumed'] > 0.001, '1_BackUpHeater_InService', '2_BackUpHeater_OutOfService')
Replace_JobStatus = {'Training or education': 'Training or Education', 'Working Full time  (>30hrs)': 'Working Full time (>30hrs)'} 
MasterDF = MasterDF.replace({"JobStatus": Replace_JobStatus}) # Few typos/repeats of the same categories - needed replacing.
FilePath3 = f"../Data/UK Data Service/Electrification of Heat Demonstration Project 2020-2022/Resampled_{SampleRate}/000MasterTable_{SampleRate}.csv"
MasterDF.to_csv(FilePath3)
LogFile.close()

In [ ]:
# PLOTTING stuff begins
#MasterDF = pd.read_csv(f"../../Data/UK Data Service/Electrification of Heat Demonstration Project 2020-2022/Resampled_{SampleRate}/000MasterTable_{SampleRate}.csv",index_col='Unnamed: 0')
FilePath4 = f"../Data/UK Data Service/Electrification of Heat Demonstration Project 2020-2022/Resampled_{SampleRate}"
PropDesignTbl = pd.read_csv("../Data/UK Data Service/Electrification of Heat Demonstration Project 2020-2022/BEIS Electrification of Heat Project - Property, Design and Installation Information.csv")
UniquePropList = MasterDF['Property_ID'].unique()
FigureLists = tqdm(UniquePropList)
X = np.arange(0,31)
#ASHPCOP = 6.08-0.09*X+0.0005*X*X
ASHPCOP = 4.08-0.09*X+0.0008*X*X
for Names in FigureLists:
    PropertyLine = PropDesignTbl.loc[(PropDesignTbl['Property_ID'] == Names)].copy()
    EPCRating,FloorArea,Employment,HouseIncome,Tenure,HPType,HP_Size_kW,Age,JobStatus,Fluid,MCS_SHLoad,MCS_Hloss = GetPropertyDetails(PropertyLine)
    FigureLists.set_description(f'Plotting ({Names}) ...')
    #
    SummaryTable1 = MasterDF.loc[(MasterDF['Property_ID'] == Names)].copy()
    Samples = SummaryTable1.shape[0]
    SummaryTable1['Timestamp'] = pd.to_datetime(SummaryTable1['Timestamp'])
    #
    H2_Median = round(SummaryTable1['COP_H2'].median(),3)
    H3_Median = round(SummaryTable1['COP_H3'].median(),3)
    H4_Median = round(SummaryTable1['COP_H4'].median(),3)
    H2_Mean = round(SummaryTable1['COP_H2'].mean(),3)
    H3_Mean = round(SummaryTable1['COP_H3'].mean(),3)
    H4_Mean = round(SummaryTable1['COP_H4'].mean(),3)
    #
    plt.figure(figsize=(48,27))
    plt.subplot(5,4,1)
    plt.title(f'Coefficient of Performance - H2 against Time ## {Names}\nSamples {Samples}, Median {H2_Median}, Mean {H2_Mean}.')
    sns.scatterplot(SummaryTable1,x="Timestamp",y="COP_H2",size=0.05,legend=False,linewidth=0)
    plt.ylim(0,10)
    plt.xticks(rotation=20)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout()
    #
    plt.subplot(5,4,2)
    plt.title(f'Coefficient of Performance - H2 against Temperature Makeup ## {Names}\nSamples {Samples}, Median {H2_Median}, Mean {H2_Mean}.')
    sns.scatterplot(SummaryTable1,x="Temperature_Makeup",y="COP_H2",size=0.05,legend=False,linewidth=0)
    sns.lineplot(x=X, y=ASHPCOP, color = 'red')
    plt.ylim(0,10)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout()
    #
    plt.subplot(5,4,3)
    plt.title(f'External Air temperature against Time ## {Names}\nSamples {Samples}.')
    sns.scatterplot(SummaryTable1,x="Timestamp",y="External_Air_Temperature",size=0.05,legend=False,linewidth=0)
    plt.ylim(-10,40)
    plt.xticks(rotation=20)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout()
    #
    plt.subplot(5,4,4)
    plt.title(f'Internal Air Temperature against Time ## {Names}\nSamples {Samples}.')
    sns.scatterplot(SummaryTable1,x="Timestamp",y="Internal_Air_Temperature",size=0.05,legend=False,linewidth=0)
    plt.ylim(10,30)
    plt.xticks(rotation=20)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout()  
    #
    plt.subplot(5,4,5)
    plt.title(f'Coefficient of Performance - H4 against Temperature Makeup ## {Names}\nSamples {Samples}, Median {H4_Median}, Mean {H4_Mean}.')
    sns.scatterplot(SummaryTable1,x="Temperature_Makeup",y="COP_H4",linewidth=0, hue="ImmersionHeater_InService_Thresh_1Wth",alpha=0.5)
    sns.lineplot(x=X, y=ASHPCOP, color = 'red')
    plt.ylim(0,10)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout() 
    #
    plt.subplot(5,4,6)
    plt.title(f'Coefficient of Performance - H4 against Temperature Makeup ## {Names}\nSamples {Samples}, Median {H4_Median}, Mean {H4_Mean}.')
    sns.scatterplot(SummaryTable1,x="Temperature_Makeup",y="COP_H4",linewidth=0, hue="BackUp_Heater_InService_Thresh_1Wth",alpha=0.5)
    sns.lineplot(x=X, y=ASHPCOP, color = 'red')
    plt.ylim(0,10)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout()
    #
    plt.subplot(5,4,7)
    plt.title(f'Energy Output over Time ## {Names} ## Samples {Samples} ## EPC rating {EPCRating}\nTotal Floor area {round(FloorArea,1)}m2, HP Size {HP_Size_kW}kW, HP Type = ({HPType}), Fluid = ({Fluid}).')
    sns.scatterplot(SummaryTable1,x="Timestamp",y="Heat_Pump_Energy_Output",size=0.05,legend=False,linewidth=0)
    sns.scatterplot(SummaryTable1,x="Timestamp",y="Whole_System_Energy_Consumed",size=0.05,legend=False,linewidth=0)
    plt.legend(labels=["HP Energy Output, thermal (kWth)","Whole System Energy Output, electric (kWe)"])
    plt.ylabel('Cummuative Energy Output over time, kWth|kWe.')
    plt.ylim(0,30000)
    plt.xticks(rotation=20)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout()    
    #
    plt.subplot(5,4,8)
    plt.title(f'Energy Output over Time ## {Names} ## Samples {Samples} ## EPC rating {EPCRating}\nTotal Floor area {round(FloorArea,1)}m2, HP Size {HP_Size_kW}kWth, HP Type = ({HPType}), Fluid = ({Fluid}).')
    sns.scatterplot(SummaryTable1,x="Timestamp",y="Hourly_Heat_Pump_Energy_Output",size=0.05,legend=False,linewidth=0,alpha=0.5)
    sns.scatterplot(SummaryTable1,x="Timestamp",y="Hourly_Heat_Pump_Power_Consumed",size=0.05,legend=False,linewidth=0,alpha=0.5)
    plt.legend(labels=["Hourly HP Output, thermal (kWth)","Hourly HP Output, electric (kWe)"])
    plt.ylabel('Energy Output over time, kWth|kWe.')
    plt.ylim(0,HP_Size_kW+2)
    plt.xticks(rotation=20)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout() 
    #
    CP_InService = SummaryTable1.loc[(SummaryTable1['HP_CircPumpInService_Thresh_1Wth'] == '1_CP_InService') & (SummaryTable1['HP_Hourly_Power_Thresh_0.1kWe'] == '1_HP_HighPower')]
    CP_OOS = SummaryTable1.loc[(SummaryTable1['HP_CircPumpInService_Thresh_1Wth'] == '2_CP_OutOfService') & (SummaryTable1['HP_Hourly_Power_Thresh_0.1kWe'] == '1_HP_HighPower')]
    Samp_IS,Samp_OOS = CP_InService.shape[0],CP_OOS.shape[0]
    plt.subplot(5,4,9)
    plt.title(f'Coefficient of Performance - H4 against Temperature Makeup ## {Names}\nSamples {Samp_IS} ## Circulating Pump In Service AND HP power greater than 0.1kWe.')
    sns.scatterplot(CP_InService,x="Temperature_Makeup",y="COP_H4",size=0.05,legend=False,linewidth=0)
    sns.lineplot(x=X, y=ASHPCOP, color = 'red')
    if Samp_IS > 10:
        XValues = CP_InService["Temperature_Makeup"]
        YValues = CP_InService["COP_H4"]
        LinearFit,coeff_A1,YIntercpt_A1,PolyFit,coeff_B1,coeff_B2,YIntercptB = GetPolyCoefficients(XValues,YValues)
        sns.lineplot(x=X,y=LinearFit, color='black',label=f'Linear = ({round(coeff_A1,4)}*X + {round(YIntercpt_A1,4)})',linewidth=5)
        sns.lineplot(x=X,y=PolyFit, color='purple',label=f'Poly = ({round(coeff_B1,4)}*X*X + {round(coeff_B2,4)}*X + {round(YIntercptB,3)})',linewidth=5)
    plt.ylim(0,10)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout()
    #
    plt.subplot(5,4,10)
    plt.title(f'Coefficient of Performance - H4 against Temperature Makeup ## {Names}\nSamples {Samp_OOS} ## Circulating Pump Out Of Service AND HP power greater than 0.1kWe.')
    sns.scatterplot(CP_OOS,x="Temperature_Makeup",y="COP_H4",size=0.05,legend=False,linewidth=0)
    sns.lineplot(x=X, y=ASHPCOP, color = 'red')
    if Samp_OOS > 10:
        XValues = CP_OOS["Temperature_Makeup"]
        YValues = CP_OOS["COP_H4"]
        LinearFit,coeff_A1,YIntercpt_A1,PolyFit,coeff_B1,coeff_B2,YIntercptB = GetPolyCoefficients(XValues,YValues)
        sns.lineplot(x=X,y=LinearFit, color='black',label=f'Linear = ({round(coeff_A1,4)}*X + {round(YIntercpt_A1,4)})',linewidth=5)
        sns.lineplot(x=X,y=PolyFit, color='purple',label=f'Poly = ({round(coeff_B1,4)}*X*X + {round(coeff_B2,4)}*X + {round(YIntercptB,3)})',linewidth=5)
    plt.ylim(0,10)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout()
    #
    plt.subplot(5,4,11)
    plt.title(f'Energy Output over Time ## {Names} ## Samples {Samples}\nEPC rating {EPCRating}, Total Floor area {round(FloorArea,1)}m2, HP Size {HP_Size_kW}kWth.')
    Slope, Intercept, R_value, P_value, Std_err = stats.linregress(SummaryTable1['Hourly_Heat_Pump_Power_Consumed'],SummaryTable1['Hourly_Heat_Pump_Energy_Output'])
    sns.regplot(data=SummaryTable1,x="Hourly_Heat_Pump_Power_Consumed", y="Hourly_Heat_Pump_Energy_Output",order=1,
                line_kws={'label':"y={0:.1f}x+{1:.1f}\nR Squared = {2:.3f}".format(Slope,Intercept,R_value*R_value)})
    plt.xlabel('Hourly_Heat_Pump_Power_Consumed, kWe.')
    plt.ylabel('Hourly_Heat_Pump_Energy_Output, kWth.')
    plt.xlim(0,HP_Size_kW/2)
    plt.ylim(0,HP_Size_kW+2)
    plt.legend()
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout()
    #
    plt.subplot(5,4,12)
    plt.title(f'Various COP performances - KDE plot ## {Names}\nSamples {Samples}, Age = {Age} years, House Income = (£{HouseIncome}). Job = ({JobStatus}).')
    sns.kdeplot(data=SummaryTable1,x="COP_H2")
    sns.kdeplot(data=SummaryTable1,x="COP_H3")
    sns.kdeplot(data=SummaryTable1,x="COP_H4")
    plt.legend(labels=["COP_H2","COP_H3","COP_H4"])
    plt.xlabel("Coefficient of Performance. (COP_H2, COP_H3, COP_H4)")
    plt.xlim(1,7)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout()
    #
    plt.subplot(5,4,13)
    plt.title(f'Coefficient of Performance - H4 against Temperature Makeup ## {Names}\nSamples {Samples}, Median {H4_Median}, Mean {H4_Mean}.')
    sns.scatterplot(SummaryTable1,x="Temperature_Makeup",y="COP_H4",linewidth=0, hue="HP_Hourly_Power_Thresh_0.1kWe",alpha=0.5)
    sns.lineplot(x=X, y=ASHPCOP, color = 'red')
    plt.ylim(0,10)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout() 
    #
    plt.subplot(5,4,14)
    plt.title(f'Coefficient of Performance - H4 against Temperature Makeup ## {Names}\nSamples {Samples}, Median {H4_Median}, Mean {H4_Mean}.')
    sns.scatterplot(SummaryTable1,x="Temperature_Makeup",y="COP_H4",linewidth=0, hue="HP_CircPumpInService_Thresh_1Wth",alpha=0.5)
    sns.lineplot(x=X, y=ASHPCOP, color = 'red')
    plt.ylim(0,10)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout()
    #
    plt.subplot(5,4,15)
    plt.title(f'Coefficient of Performance - H4 against Temperature Makeup ## {Names}\nSamples {Samples}, Median {H4_Median}, Mean {H4_Mean}.')
    sns.scatterplot(SummaryTable1,x="Temperature_Makeup",y="COP_H4",linewidth=0, hue="Heat_Pump_Return_Temperature_Switch",alpha=0.5)
    sns.lineplot(x=X, y=ASHPCOP, color = 'red')
    plt.ylim(0,10)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout()
    #
    plt.subplot(5,4,16)
    plt.title(f'Coefficient of Performance - H4 against Temperature Makeup ## {Names}\nSamples {Samples}, Median {H4_Median}, Mean {H4_Mean}.')
    sns.scatterplot(SummaryTable1,x="Temperature_Makeup",y="COP_H4",linewidth=0, hue="Hot_Water_Flow_Temperature_Switch",alpha=0.5)
    sns.lineplot(x=X, y=ASHPCOP, color = 'red')
    plt.ylim(0,10)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout()    
    #
    plt.subplot(5,4,17)
    plt.title(f'Coefficient of Performance - H4 against Temperature Makeup ## {Names}\nSamples {Samples}, Median {H4_Median}, Mean {H4_Mean}.')
    sns.scatterplot(SummaryTable1,x="Timestamp",y="COP_H4",linewidth=0, hue="HP_Hourly_Power_Thresh_0.1kWe",alpha=0.5)
    plt.ylim(0,10)
    plt.xticks(rotation=20)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout() 
    #
    plt.subplot(5,4,18)
    plt.title(f'Coefficient of Performance - H4 against Temperature Makeup ## {Names}\nSamples {Samples}, Median {H4_Median}, Mean {H4_Mean}.')
    sns.scatterplot(SummaryTable1,x="Timestamp",y="COP_H4",linewidth=0, hue="HP_CircPumpInService_Thresh_1Wth",alpha=0.5)
    plt.ylim(0,10)
    plt.xticks(rotation=20)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout()
    #
    plt.subplot(5,4,19)
    plt.title(f'Coefficient of Performance - H4 against Temperature Makeup ## {Names}\nSamples {Samples}, Median {H4_Median}, Mean {H4_Mean}.')
    sns.scatterplot(SummaryTable1,x="Timestamp",y="COP_H4",linewidth=0, hue="Heat_Pump_Return_Temperature_Switch",alpha=0.5)
    plt.ylim(0,10)
    plt.xticks(rotation=20)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout()
    #
    plt.subplot(5,4,20)
    plt.title(f'Coefficient of Performance - H4 against Temperature Makeup ## {Names}\nSamples {Samples}, Median {H4_Median}, Mean {H4_Mean}.')
    sns.scatterplot(SummaryTable1,x="Timestamp",y="COP_H4",linewidth=0, hue="Hot_Water_Flow_Temperature_Switch",alpha=0.5)
    plt.ylim(0,10)
    plt.xticks(rotation=20)
    plt.grid(which='both',linewidth = 0.5)
    plt.tight_layout()   
    #    
    plt.savefig(f"{FilePath4}/Plots/{EPCRating}_{Names}.png")
    plt.close('all')
